# Perform Sample Searches on Vector Database 

## Introduction

This notebook provides an optional exercise for exploring document search within a vector database. Experiment with different search techniques using Elasticsearch or Milvus as your chosen database.

This notebook demonstrates how to perform sample searches using various techniques and Analyze the search results to understand the effectiveness of different methods.
While this notebook is optional, it can be valuable for gaining hands-on experience with vector databases and document search techniques. If you prefer, you can skip this exercise and proceed directly to `Create and Deploy QnA Python Function`

In this notebook, we will cover the following actions:

- Establishing a connection to the chosen vector database (Elasticsearch or Milvus).
- If your connection type is Elastic Search 
    - Using an Elastic Learned Sparse Encoder (ELSER) and LangChain to search and retrieve relevant documents based on specific queries in Elastic Search.  
- If your connection type is Milvus    
    - Employing an embedding model and LangChain to search and retrieve relevant documents based on specific queries in Milvus.


## Contents

This notebook contains the following parts:
- [Setup](#setup)
- [Connect to Vector Database](#connect)
- [Q&A using Vectorstore and query templates](#QnATest)




Below cell installs specific libraries required to run this notebook

<a id="setup"></a>
### Pre-Requisite Libraries and Dependencies
Download and import mandatory libraries and dependencies

In [ ]:

!pip install elasticsearch | tail -n 1
!pip install langchain | tail -n 1
!pip install ibm_watsonx_ai==1.1.6 | tail -n 1
!pip install langchain_elasticsearch | tail -n 1
!pip install langchain_milvus | tail -n 1
!pip install pymilvus | tail -n 1

Restart the kernel after performing the pip install if the below cell fails to import all the libraries.

In [ ]:
from elasticsearch import Elasticsearch, helpers
from ibm_watsonx_ai import Credentials
from ibm_watsonx_ai.foundation_models import Embeddings
from ibm_watsonx_ai.metanames import EmbedTextParamsMetaNames as EmbedParams
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
from ibm_watsonx_ai import APIClient
import json
import os
import shutil 
import warnings
warnings.filterwarnings("ignore")
from pymilvus import(Milvus,IndexType,Status,connections,FieldSchema,DataType,Collection,CollectionSchema,utility)

In [ ]:
project_id=os.environ['PROJECT_ID']
# Environment and host url
hostname = os.environ['RUNTIME_ENV_APSX_URL']

if hostname.endswith("cloud.ibm.com") == True:
    environment = "cloud"
    project_id = os.environ['PROJECT_ID']
else:
    environment = "on-prem"
    from ibm_watson_studio_lib import access_project_or_space
    wslib = access_project_or_space()
   




<a id="parameterimport"></a>
### Import Parameter Sets, Credentials and Helper functions script.

Below cells imports parameter sets values, the credentials and helper functions script


In [ ]:
parameter_sets = ["RAG_parameter_set","RAG_advanced_parameter_set"]
parameters={}
for parameter_set in parameter_sets:
    parameter_vals= wslib.assets.get_asset(parameter_set, "parameter_set", raw=True)
    if 'entity' in parameter_vals and 'parameter_set' in parameter_vals['entity'] and 'parameters' in parameter_vals['entity']['parameter_set']:
        parameter_vals = parameter_vals['entity']
    params = {param['name']: param['value'] for param in parameter_vals['parameter_set']['parameters']}
    parameters.update(params)    

In [ ]:
ibm_api_key=parameters['watsonx_ai_api_key']
if environment == "cloud":
    WML_SERVICE_URL = f"https://{hostname.split('.')[1]}.ml.cloud.ibm.com" if hostname.split('.')[0].endswith("api") and hostname.split('.')[2] == "dataplatform" else "https://us-south.ml.cloud.ibm.com"
    wml_credentials = {"apikey": ibm_api_key, "url": WML_SERVICE_URL}
else:
    token = os.environ['USER_ACCESS_TOKEN']
    wml_credentials = {"token": token,"instance_id" : "openshift","url": hostname}

In [ ]:
try:
    filename = 'rag_helper_functions.py'
    wslib.download_file(filename)
    import rag_helper_functions
    print("rag_helper_functions imported from the project assets")
except NameError as e:
    print(str(e))
    print("If running watsonx.ai aaS on IBM Cloud, check that the first cell in the notebook contains a project token. If not, select the vertical ellipsis button from the notebook toolbar and `insert project token`. Also check that you have specified your ibm_api_key in the second code cell of the notebook")


### Set Watsonx.ai client
Below cell uses the watson machine learning credentials to create an API client to interact with the project and deployment space. 

In [ ]:
client = APIClient(wml_credentials)
client.set.default_project(project_id=project_id)


<a id="connect"></a>
### Connecting to a vector database

#### Connecting using Project Connection Asset (default)
The notebook, by default, will look for a connection asset in the project named `milvus_connect` or `elasticsearch_connect`.  You can set this up by following the instructions in the project readme. 
This code checks if a specified connection exists in the project. If found, it retrieves the connection details and identifies the connection type. Depending on the connection type, it establishes a connection to the appropriate database. If the connection is not found, it raises an error indicating the absence of the specified connection in the project.



In [ ]:
connection_name=parameters["connection_asset"]
if(next((conn for conn in wslib.list_connections() if conn['name'] == connection_name), None)):
    print(connection_name, "Connection found in the project")
    db_connection = wslib.get_connection(connection_name)
    
    connection_datatypesource_id=client.connections.get_details(db_connection['.']['asset_id'])['entity']['datasource_type']
    connection_type = client.connections.get_datasource_type_details_by_id(connection_datatypesource_id)['entity']['name']
    
    print("Successfully retrieved the connection details")
    print("Connection type is identified as:",connection_type)
    if connection_type=="elasticsearch":
        es_client=rag_helper_functions.create_elastic_client(db_connection)
    elif connection_type=="milvus":
        db_connection['database']=db_connection['database'] if 'database' in db_connection else 'default'

        connection_params = {
            'database': db_connection['database'],
            'host': db_connection['host'],
            'port': db_connection['port'],
            'user': db_connection['username'],
            'password': db_connection['password'],
            'secure': True
        }
        if 'ssl_certificate' in db_connection :
            ssl_certificate_content = db_connection.get('ssl_certificate') if db_connection.get('ssl_certificate') else ""
            cert_file_path = 'milvus_conn.cert'
            with open(cert_file_path, 'w') as file:
                file.write(ssl_certificate_content)
            connection_params['server_pem_path'] = cert_file_path
        milvus_client = connections.connect(**connection_params)
        print("Successfully connected to milvus database")
else:
    db_connection=""
    raise ValueError(f"No connection named {connection_name} found in the project.")

<a id="QnATest"></a>
## Q&A on the Vector Database Index/Collection

The following sections of the notebook are designed to test a sample Question and Answer (QnA) interaction on the vector store. The subsequent cell in the notebook executes this test and provides a response that includes several key pieces of information. 



In [ ]:
question ="how to configure Decision Optimization experiment?"

### 1. Using Elastic Search Query Templates on Elastic Search Vector Database

The following section of the notebook is designed to test a sample Question and Answer (QnA) interaction using sample template of ELSER model, assuming it is utilized.
This response comprises of:

* `Relevance Score`: A numerical value indicating the relevance or confidence level of the answer provided by the model.
* `Title`: The title of the document from which the answer is derived.
* `Document ID`: A unique identifier for the document within the database or index.
* `Document URL`: The location where the original document can be accessed or referenced.
* `Document Content`: The actual content or text from the document that is relevant to the queried question.
* `Source`: The source of the content or text for the relevant document queried. 
* `Page Number`: Page Number of the document (if applicable).

This setup allows for a practical demonstration of the model's capabilities in retrieving and presenting information in response to a specific query. 
There are 2 ways to perform this step, depending on the `elastic_search_template_file` parameter provided in the parameter set by the user. 
1. **ELSER**: An ELSER exclusive search query is invoked.
2. **ELSER + BM25**: A hybrid search query that is a combination of a tradition vector search and ELSER is invoked.


In [ ]:

if connection_type=="elasticsearch":
    wslib.download_file(parameters['elastic_search_template_file'])
    with open(parameters['elastic_search_template_file']) as f:
        es_query = json.load(f)

    print("updating tokens and query fields")

    if "BM25" in parameters['elastic_search_template_file']:
        es_query['bool']['should'][0]['match']['description']['query'] = question
        es_query['bool']['should'][1]['text_expansion']['vector.tokens']['model_id'] = parameters['elastic_search_model_id']
        es_query['bool']['should'][1]['text_expansion']['vector.tokens']['model_text'] = question
        print("ELSER + BM25 Lexical search query : ")
    else:
        es_query['text_expansion']['vector.tokens']['model_id'] = parameters['elastic_search_model_id']
        es_query['text_expansion']['vector.tokens']['model_text'] = question
        print("ELSER search query : ")

    print(es_query)


    try:
        response = es_client.search(
            index=parameters["vector_store_index_name"], 
            size=parameters['vectorsearch_top_n_results'],
            query=es_query
        )
        print("\nResponse:")
        for hit in response['hits']['hits']:

            score = hit['_score']
            title = hit['_source']['metadata']['title']
            page_content=hit['_source']['text']
            source = hit['_source']['metadata']['source']
            url = hit['_source']['metadata']['document_url']
            page_number = hit['_source']['metadata']['page_number']


            print(f"\nRelevance Score  : {score}\nTitle            : {title}\nSource     : {source}\nDocument Content : {page_content}\nDocument URL : {url}\nPage Number : {page_number}")

    except Exception as e:
            print("An error occurred while querying elastic search, please retry after sometime:", e)




### 2. Using Milvus Search option using pymilvus

Below code initializes a Milvus collection using a specified collection name and loads it into memory. It then generates embeddings for a given question using above specified embedding model. The code performs a vector search in the collection based on these embeddings, and retrieves the top 5 closest results. 

In [ ]:
if connection_type=="milvus":
    if environment=="cloud":
        credentials=Credentials(
            api_key = parameters['watsonx_ai_api_key'],
            url =WML_SERVICE_URL)
        embedding = Embeddings(
          model_id=parameters['embedding_model_id'],
          credentials=credentials,
          project_id=project_id,
          verify=True
        )
    elif environment=="on-prem":
        print("Downloading the embedding model from HuggingFace")
        from langchain_community.embeddings import HuggingFaceEmbeddings
        embedding=HuggingFaceEmbeddings(model_name=parameters['embedding_model_id'])
        

    index_params = {"index_type": "HNSW","metric_type": "L2",
                        "params": {"M": 16,"efConstruction": 200,"efSearch": 16}}
    
    collection = Collection(parameters["vector_store_index_name"])
    collection.load()
    query_embeddings = embedding.embed_documents(texts=[question])

    results = collection.search(data=query_embeddings,anns_field="vector", param=index_params,limit=5,  expr=None,  output_fields=['text'])
    for res in results[0]:
        print(res.to_dict())

### 3. Using the Langchain Vector Store
Below code utilizes Langchains vector store extension to retrieve documents. <br>

The code sets up a vector store based on the specified connection type, either "elasticsearch" or "milvus". <br>

* If `connection_type` is `"elasticsearch"`, it imports `ElasticsearchStore` from the `langchain_elasticsearch` library and initializes it with an Elasticsearch client and specified parameters with the given model ID.
* If `connection_type` is `"milvus"`, the code imports `langchain_milvus` and the `Milvus` vector store is then created using the embedding function, connection parameters, and index settings. 

Below cell should run successfully, regardless of which vector database is used.


In [ ]:


if connection_type=="elasticsearch":
    from langchain_elasticsearch import ElasticsearchStore
    vector_store=ElasticsearchStore(
                    es_connection=es_client,
                    index_name=parameters["vector_store_index_name"],
                    strategy=ElasticsearchStore.SparseVectorRetrievalStrategy(model_id=parameters['elastic_search_model_id'])
                    )
    print("Elastic Search Vector Store Created with",parameters['elastic_search_model_id'])
elif connection_type=="milvus":
    from langchain_milvus import Milvus
    db_connection['database'] =db_connection.get('database','default')
    milvus_credentials={'database': db_connection['database'],'password':db_connection['password'] ,'port': db_connection['port'] ,'host': db_connection['host'],"secure": True,'user': db_connection['username']}
    vector_store = Milvus(
        embedding_function=embedding,
        connection_args=milvus_credentials,
        index_params=index_params,
        primary_field='id',
        #text_field="page_content",
        collection_name=parameters["vector_store_index_name"] 
         
    )
    print("Milvus Vector Store Created")

if connection_type=="elasticsearch":
    results_and_scores=vector_store.similarity_search_with_relevance_scores(question, k=parameters['vectorsearch_top_n_results'])
elif connection_type=="milvus":
    results_and_scores = vector_store.similarity_search_with_score_by_vector(embedding.embed_query(question), k=parameters['vectorsearch_top_n_results'])  # k is the number of results you want

print("Search results from",connection_type,"vectorstore:")
results_and_scores

Proceed to **Create and Deploy QnA Python Function** notebook to create and deploy the RAG python function.



**Sample Materials, provided under license.</a> <br>
Licensed Materials - Property of IBM. <br>
© Copyright IBM Corp. 2024. All Rights Reserved. <br>
US Government Users Restricted Rights - Use, duplication or disclosure restricted by GSA ADP Schedule Contract with IBM Corp. <br>**
